# Machine Learning Project: French Grand Debat

### Project Introduction:

First, we would like to choose a problematic according to the data we collected. The aim of this project could be to answer the following questions:

    - What are the 5 most important topic on each theme?
  
    - Can we build different profiles of people with their ideas on the 4 themes?
 




In [2]:
# librairies imported
import src.utils as ut #read data

#### Data extraction

In [3]:
df_fiscalite = ut.read_data('data/LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES.json')
df_democratie = ut.read_data('data/DEMOCRATIE_ET_CITOYENNETE.json')
df_ecologie = ut.read_data('data/LA_TRANSITION_ECOLOGIQUE.json')
df_organisation = ut.read_data('data/ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS.json')

In [11]:
print(df_fiscalite.loc[0, 'authorId'])
print(df_fiscalite.loc[1, 'authorId'])

VXNlcjo3ZTVjYTUwMi0xZDZlLTExZTktOTRkMi1mYTE2M2VlYjExZTE=
VXNlcjo5NmNhYWM4ZS0xZTIwLTExZTktOTRkMi1mYTE2M2VlYjExZTE=
